# 🩺 **MAGFLOW**

> Análisis comparativo de los estudios de resonancia de flujo realizados a 20 pacientes con implante de vávlula áortica transcatéter (TAVI).

Este estudio evalúa los siguientes parámetros biomecánicos durante el ciclo cardíaco completo:

- **Flujo volumétrico** (*Q*) - Volumen de sangre por unidad de tiempo [`m³/s`]
- **Velocidad máxima** (*v*<sub>max</sub>) - Velocidad pico en las secciones aórticas [`m/s`]
- **TAWSS** (*Time-Averaged Wall Shear Stress*) - Esfuerzo cortante promedio [`Pa`]
- **OSI** (*Oscillatory Shear Index*) - Índice de oscilación del esfuerzo cortante [`adimensional`]
- **Pérdida de carga** - Disipación de energía en el flujo [`Pa`]
- **Gradientes de presión** - Variación espacial de la presión [`Pa/m`]
- **Vorticidad** - Medida de la rotación local del fluido [`s⁻¹`]
- **Helicidad** - Correlación entre velocidad y vorticidad [`m²/s²`]

## **⚙️ Configuración**

---

Se importan las bibliotecas especializadas y módulos desarrollados específicamente para el procesamiento de datos de resonancia magnética de flujo 4D.

**Bibliotecas principales:**
- **NumPy/SciPy**: Cálculos numéricos y álgebra lineal
- **PyVista**: Visualización y procesamiento de mallas 3D
- **Matplotlib**: Generación de gráficos y análisis estadístico
- **TQDM**: Barras de progreso para seguimiento de procesos

In [ ]:
# Standard library imports
import importlib
import math
from pathlib import Path

import matplotlib.pyplot as plt

# Scientific computing libraries
import pyvista as pv
from tqdm.notebook import tqdm

# Internal specialized modules
import magflow.utils.energy as ene
import magflow.utils.flow as flow
import magflow.utils.helicity as hel
import magflow.utils.loader as load
import magflow.utils.osi as osi
import magflow.utils.pressure as pre
import magflow.utils.velocity as vel
import magflow.utils.visualization as viz
import magflow.utils.vorticity as vor
import magflow.utils.wss as wss


La configuración establece los parámetros fundamentales para garantizar la reproducibilidad y consistencia del análisis de los pacientes.

In [ ]:
# Jupyter and PyVista configuration
%matplotlib inline
pv.set_jupyter_backend("static")
pv.global_theme.allow_empty_mesh = True
pv.global_theme.window_size = [1024, 768]
pv.global_theme.font.size = 12

# Physical constants for blood flow analysis
BLOOD_DENSITY = 1050.0  # kg/m³ - Blood density
BLOOD_VISCOSITY = 0.004  # Pa·s - Blood dynamic viscosity

# Directory configuration
ASSETS_DIR = Path(r"C:\Users\Luis\Projects\Magflow")
CACHE_DIR = Path(r"C:\Users\Luis\Repositories\magflow\.cache")

# Ensure directories exist
CACHE_DIR.mkdir(exist_ok=True)

# Patient configuration
PATIENT_IDS = [
    "MF033",
    "MF051",
    "MF056",
    "MF057",
    "MF064",
    "MF066",
    "MF069",
    "MF075",
    "MF121",
    "MF126",
    "MF132",
    "MF138",
    "MF146",
]
DEFAULT_PATIENT = "MF033"  # For single patient analysis

# Analysis parameters
NUM_CENTRELINE_POINTS = 24
CROSS_SECTION_PERCENTAGE = 25

# Visualization configuration
COLOR_MAP = [
    "blue",
    "red",
    "green",
    "orange",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
    "magenta",
    "yellow",
    "black",
    "navy",
    "maroon",
    "lime",
    "aqua",
    "teal",
    "silver",
    "fuchsia",
]

# Display configuration summary
print("\n🔧 CONFIGURATION SUMMARY")
print("-" * 40)
print(f"📁 Assets directory: {ASSETS_DIR}")
print(f"💾 Cache directory: {CACHE_DIR}")
print(f"👥 Number of patients: {len(PATIENT_IDS)}")
print(f"🎯 Default patient: {DEFAULT_PATIENT}")
print(f"📏 Centreline points: {NUM_CENTRELINE_POINTS}")
print(f"🔍 Cross-section analysis: {CROSS_SECTION_PERCENTAGE}%")
print(f"🩸 Blood density: {BLOOD_DENSITY} kg/m³")
print(f"🌊 Blood viscosity: {BLOOD_VISCOSITY} Pa·s")

## **🏗️ Carga de datos**

---

Carga e inicialización de todos los conjuntos de datos necesarios para el análisis del flujo aórtico.

Se importan los datos de velocidad del flujo sanguíneo almacenados en formato VTK estructurado (`*.vts`), obtenidos mediante resonancia magnética de flujo 4D. Adicionalmente, se carga la geometría aórtica tridimensional previamente segmentada en *3D Slicer*.

**Proceso de carga detallado:**

- **Datos de velocidad**: Campos vectoriales 3D del flujo sanguíneo para cada instante temporal
- **Geometría aórtica**: Biomodelo tridimensional segmentado de la aorta ascendente
- **Línea central**: Trayectoria anatómica de referencia con puntos distribuidos uniformemente
- **Validación de integridad**: Verificación de la consistencia y completitud de los datos
- **Análisis temporal**: Identificación de timesteps disponibles y sincronización entre pacientes

**Optimización del rendimiento:**
- **Sistema de caché**: Almacenamiento de métricas calculadas para evitar recálculos
- **Carga paralela**: Procesamiento simultáneo de múltiples pacientes cuando sea posible
- **Validación previa**: Verificación de la disponibilidad de archivos antes del procesamiento

El sistema genera un resumen completo con estadísticas del conjunto de datos, métricas de calidad y diagnósticos de posibles problemas de integridad.

In [ ]:
importlib.reload(load)

all_patient_data = {}
all_timesteps = set()

for patient_id in tqdm(PATIENT_IDS, desc="Loading patients"):
    print(f"Loading patient: {patient_id}")
    patient_data = load.load_patient(patient_id, ASSETS_DIR, NUM_CENTRELINE_POINTS)

    if patient_data["timesteps"]:
        all_patient_data[patient_id] = patient_data
        all_timesteps.update(patient_data["timesteps"].keys())
        print(
            f"Successfully loaded patient {patient_id} with {len(patient_data['timesteps'])} timesteps"
        )
    else:
        print(f"Warning: No data loaded for patient {patient_id}")

sorted_timesteps = sorted(all_timesteps)

# Print summary
print("\nDATA LOADING SUMMARY")
print("-" * 30)
print(f"Total patients loaded: {len(all_patient_data)}")
print(f"Unique timesteps across all patients: {len(sorted_timesteps)}")
print(
    f"Timestep range: {min(sorted_timesteps) if sorted_timesteps else 'N/A'} - {max(sorted_timesteps) if sorted_timesteps else 'N/A'}"
)

In [ ]:
importlib.reload(viz)

# Setup
n_patients = len(all_patient_data)
n_cols = min(5, n_patients)
n_rows = math.ceil(n_patients / n_cols)

# Find common timestep
selected_ts = 0
plotter = pv.Plotter(shape=(n_rows, n_cols), notebook=True)

# Process each patient
for idx, patient_id in enumerate(PATIENT_IDS):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Calculate subplot position
    row = idx // n_cols
    col = idx % n_cols

    # Add patient to subplot
    ts_used = viz.render_patient(
        plotter, patient_id, all_patient_data[patient_id], row, col, selected_ts
    )

# Show the visualization
plotter.show()

## 💧 **Flujo volumétrico**

El flujo volumétrico representa el volumen de sangre que atraviesa una sección transversal de la aorta por unidad de tiempo. Este parámetro hemodinámico fundamental permite evaluar la función cardiovascular y detectar alteraciones en el flujo sanguíneo tras el implante de válvula aórtica transcatéter (TAVI).

El cálculo se basa en la integración del campo de velocidades perpendicular a la superficie de cada sección transversal:

$$
Q = \int_A \vec{v} \cdot \hat{n} \, dA
$$

**Parámetros de la ecuación:**
- *Q* = flujo volumétrico [m³/s]
- $\vec{v}$ = vector velocidad del flujo en cada punto
- $\hat{n}$ = vector normal unitario perpendicular a la sección
- $A$ = área de la sección transversal aórtica

**Metodología de análisis:**

1. **Generación de secciones transversales**: Se crean múltiples planos perpendiculares distribuidos uniformemente a lo largo de la línea central aórtica
2. **Análisis temporal completo**: Los cálculos se ejecutan para cada instante del ciclo cardíaco registrado
3. **Promedio espacial**: Se calcula el valor medio entre todas las secciones para obtener medidas comparables entre pacientes

In [ ]:
importlib.reload(flow)

# Calculate flow rates
cross_section_index = flow.get_cross_section_index(
    CROSS_SECTION_PERCENTAGE, NUM_CENTRELINE_POINTS
)

# Initialize multi-patient flow rates dictionary
all_flow_rates = {}

# Calculate flow rates for all patients
for patient_id in tqdm(PATIENT_IDS, desc="Calculating flow rates"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached flow rates (metrics only)
    cached_flow_rates = load.load_metric_cache(patient_id, "flow_rates", CACHE_DIR)
    if cached_flow_rates is not None:
        all_flow_rates[patient_id] = cached_flow_rates
        continue

    # Get patient-specific data
    patient_timesteps = all_patient_data[patient_id]["timesteps"]
    patient_biomodel = all_patient_data[patient_id]["biomodel"]
    patient_centreline = all_patient_data[patient_id]["centerline"].points

    # Calculate the specific point along the centreline for this patient
    selected_centreline_point = patient_centreline[cross_section_index]

    flow_rates = flow.calculate_flow_rates(
        patient_id,
        patient_timesteps,
        patient_biomodel,
        patient_centreline,
        selected_centreline_point,
    )

    all_flow_rates[patient_id] = flow_rates

    # Cache only the computed metrics
    load.save_metric_cache(patient_id, "flow_rates", flow_rates, CACHE_DIR)


In [ ]:
# Create multi-patient flow rate comparison plot
n_patients = len(all_flow_rates)
if n_patients == 0:
    print("No flow rate data available for plotting")
else:
    # Calculate optimal grid dimensions using the function
    n_rows, n_cols = viz.calculate_subplot_grid(n_patients)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
    fig.suptitle(
        f"Flow Rate Comparison at Cross-Section {cross_section_index} ({CROSS_SECTION_PERCENTAGE}%)",
        fontsize=16,
        fontweight="bold",
    )

    # Handle single subplot case and flatten axes for easier indexing
    axes = [axes] if n_patients == 1 else axes.flatten()
    patient_colors = dict(zip(PATIENT_IDS, COLOR_MAP[: len(PATIENT_IDS)], strict=False))

    # Plot individual patients on subplots
    for idx, patient_id in enumerate(PATIENT_IDS):
        if patient_id in all_flow_rates and idx < len(axes):
            flow.plot_flow_rates(
                axes[idx],
                patient_id,
                all_flow_rates[patient_id],
                patient_colors[patient_id],
            )

    # Hide unused subplots
    for idx in range(len(all_flow_rates), len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.show()

    # Create comprehensive comparison plot with overlay of all patients
    flow.plot_comparison(PATIENT_IDS, all_flow_rates, patient_colors)

    # Get comprehensive statistics
    flow_rate_statistics = flow.get_flow_rate_statistics(all_flow_rates)

## 🚀 **Velocidad máxima**

La velocidad máxima (*v*<sub>max</sub>) representa el valor pico de velocidad del flujo sanguíneo registrado en cada sección transversal aórtica durante el ciclo cardíaco.

**Definición matemática:**

$$
v_{max} = \max(\vec{v} \cdot \hat{n})
$$

Donde:
- *v*<sub>max</sub> = velocidad máxima en la sección [m/s]
- $\vec{v}$ = vector velocidad en cada punto [m/s]
- $\hat{n}$ = vector normal unitario a la sección

**Metodología de cálculo:**

1. **Extracción de velocidades**: Se obtienen las componentes normales de velocidad en cada punto de las secciones transversales
2. **Identificación del máximo**: Se determina el valor pico de velocidad para cada sección y timestep
3. **Análisis temporal**: Se evalúa la evolución de la velocidad máxima a lo largo del ciclo cardíaco

In [ ]:
importlib.reload(vel)

# Multi-patient velocity analysis
all_max_velocities = {}
all_mean_velocities = {}

# Process each patient
for patient_id in tqdm(PATIENT_IDS, desc="Processing patients"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached velocity metrics
    cached_max_vel = load.load_metric_cache(patient_id, "max_velocities", CACHE_DIR)
    cached_mean_vel = load.load_metric_cache(patient_id, "mean_velocities", CACHE_DIR)

    if cached_max_vel is not None and cached_mean_vel is not None:
        all_max_velocities[patient_id] = cached_max_vel
        all_mean_velocities[patient_id] = cached_mean_vel
        continue

    # Calculate velocities if not cached
    max_vels, mean_vels = vel.calculate_velocities(
        patient_id, all_patient_data[patient_id]
    )

    # Store in analysis dictionaries
    all_max_velocities[patient_id] = max_vels
    all_mean_velocities[patient_id] = mean_vels

    # Cache the computed metrics individually
    load.save_metric_cache(patient_id, "max_velocities", max_vels, CACHE_DIR)
    load.save_metric_cache(patient_id, "mean_velocities", mean_vels, CACHE_DIR)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Multi-Patient Velocity Analysis", fontsize=16, fontweight="bold")

# Plot 1: Maximum velocities for all patients
vel.plot_velocity_time_series(
    axes[0, 0],
    PATIENT_IDS,
    all_max_velocities,
    patient_colors,
    "Maximum Velocity by Patient",
    "Maximum Velocity (mm/s)",
    "max",
)

# Plot 2: Mean velocities for all patients
vel.plot_velocity_time_series(
    axes[0, 1],
    PATIENT_IDS,
    all_mean_velocities,
    patient_colors,
    "Mean Velocity by Patient",
    "Mean Velocity (mm/s)",
    "mean",
)

# Plot 3: Peak maximum velocity comparison
vel.plot_velocity_comparison_bars(
    axes[1, 0],
    PATIENT_IDS,
    all_max_velocities,
    patient_colors,
    "Peak Maximum Velocity Comparison",
    "Peak Maximum Velocity (mm/s)",
    "peak",
)

# Plot 4: Average maximum velocity comparison
vel.plot_velocity_comparison_bars(
    axes[1, 1],
    PATIENT_IDS,
    all_max_velocities,
    patient_colors,
    "Average Maximum Velocity Comparison",
    "Average Maximum Velocity (mm/s)",
    "average",
)

plt.tight_layout()

# # Calculate and print statistics
# patient_stats, cross_patient_stats = vel.calculate_velocity_statistics(
#     PATIENT_IDS, all_max_velocities, all_mean_velocities
# )

# vel.print_velocity_summary(patient_stats, cross_patient_stats)

## 🌊 **Esfuerzo parietal (WSS)**

El esfuerzo cortante parietal (*Wall Shear Stress*, WSS) representa la tensión tangencial que ejerce el flujo sanguíneo sobre la pared arterial. Este parámetro biomecánico es fundamental para evaluar el riesgo de aterosclerosis y la integridad endotelial post-TAVI.

**Definición matemática:**

Para un flujo cerca de la pared arterial, el WSS se define como:

$$
\tau_w = \mu \left. \frac{\partial u}{\partial n} \right|_{pared}
$$

**Donde:**
- $\tau_w$ = esfuerzo cortante parietal [Pa]
- $\mu$ = viscosidad dinámica de la sangre (≈ 0.004 Pa·s)
- $\frac{\partial u}{\partial n}$ = gradiente de velocidad normal a la pared arterial [s⁻¹]

**Significado clínico:**

- **WSS bajo (< 0.4 Pa)**: Asociado con activación endotelial, inflamación y aterogénesis
- **WSS fisiológico (0.4-1.5 Pa)**: Rango normal que mantiene la función endotelial
- **WSS elevado (> 1.5 Pa)**: Puede indicar estenosis o turbulencia local

**Metodología de cálculo:**

1. **Identificación de la superficie**: Se utiliza la geometría aórtica segmentada como superficie de pared
2. **Cálculo de gradientes**: Se determinan los gradientes de velocidad perpendiculares a la superficie
3. **Análisis temporal**: Se evalúa la evolución del WSS durante todo el ciclo cardíaco

Este análisis permite identificar regiones de riesgo aterosclerótico y evaluar el impacto hemodinámico del implante valvular transcatéter.

In [ ]:
importlib.reload(wss)

# Parameters
mu = 0.004  # Blood viscosity in Pa·s (typical value)

all_wss_values = {}
all_max_wss_values = {}
all_avg_wss_values = {}

for patient_id in tqdm(PATIENT_IDS, desc="Calculating WSS"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached WSS metrics
    cached_wss = load.load_metric_cache(patient_id, "wss_values", CACHE_DIR)
    cached_max_wss = load.load_metric_cache(patient_id, "max_wss_values", CACHE_DIR)
    cached_avg_wss = load.load_metric_cache(patient_id, "avg_wss_values", CACHE_DIR)

    if (
        cached_wss is not None
        and cached_max_wss is not None
        and cached_avg_wss is not None
    ):
        all_wss_values[patient_id] = cached_wss
        all_max_wss_values[patient_id] = cached_max_wss
        all_avg_wss_values[patient_id] = cached_avg_wss
        continue

    # Calculate WSS if not cached
    wss_vals, max_wss_vals, avg_wss_vals = wss.calculate_patient_wss(
        patient_id, all_patient_data[patient_id], mu
    )

    # Store in analysis dictionaries
    all_wss_values[patient_id] = wss_vals
    all_max_wss_values[patient_id] = max_wss_vals
    all_avg_wss_values[patient_id] = avg_wss_vals

    # Cache the computed metrics individually
    load.save_metric_cache(patient_id, "wss_values", wss_vals, CACHE_DIR)
    load.save_metric_cache(patient_id, "max_wss_values", max_wss_vals, CACHE_DIR)
    load.save_metric_cache(patient_id, "avg_wss_values", avg_wss_vals, CACHE_DIR)

In [ ]:
# Main 2x2 subplot figure
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Multi-Patient Wall Shear Stress Analysis", fontsize=16, fontweight="bold")

# Plot 1: Maximum WSS time series
wss.plot_wss_time_series(
    axes[0, 0],
    PATIENT_IDS,
    all_max_wss_values,
    patient_colors,
    "Maximum Wall Shear Stress by Patient",
    "Maximum WSS (Pa)",
    "max",
)

# Plot 2: Average WSS time series
wss.plot_wss_time_series(
    axes[0, 1],
    PATIENT_IDS,
    all_avg_wss_values,
    patient_colors,
    "Average WSS by Patient",
    "Average WSS (Pa)",
    "avg",
)

# Plot 3: Peak maximum WSS comparison
wss.plot_wss_comparison_bars(
    axes[1, 0],
    PATIENT_IDS,
    all_max_wss_values,
    patient_colors,
    "Peak Maximum WSS Comparison",
    "Peak Maximum WSS (Pa)",
    "peak",
)

# Plot 4: Average maximum WSS comparison
wss.plot_wss_comparison_bars(
    axes[1, 1],
    PATIENT_IDS,
    all_max_wss_values,
    patient_colors,
    "Average Maximum WSS Comparison",
    "Average Maximum WSS (Pa)",
    "average",
)

plt.tight_layout()
plt.show()

# # Calculate and print statistics
# patient_stats, cross_patient_stats = wss.calculate_wss_statistics(
#     PATIENT_IDS, all_max_wss_values, all_avg_wss_values
# )

# wss.print_wss_summary(patient_stats, cross_patient_stats)

## 🔄 **Índice oscilatorio (OSI)**

El Índice de Esfuerzo Cortante Oscilatorio (*Oscillatory Shear Index*, OSI) cuantifica la variabilidad direccional del esfuerzo cortante parietal durante el ciclo cardíaco. Este parámetro es fundamental para identificar regiones con riesgo elevado de aterogénesis y disfunción endotelial.

**Definición matemática:**

$$
OSI = \frac{1}{2} \left( 1 - \frac{\left| \int_0^T \vec{\tau_w} \, dt \right|}{\int_0^T \left| \vec{\tau_w} \right| \, dt} \right)
$$

**Donde:**
- *OSI* = índice de esfuerzo cortante oscilatorio [adimensional]
- $\vec{\tau_w}$ = vector de esfuerzo cortante parietal [Pa]
- *T* = período del ciclo cardíaco [s]

**Interpretación clínica:**

- **OSI ≈ 0**: Flujo unidireccional estable, condiciones hemodinámicas favorables
- **OSI ≈ 0.5**: Flujo altamente oscilatorio, riesgo aterogénico máximo
- **OSI > 0.2**: Umbral asociado con activación endotelial y formación de placas

**Significado biomecánico:**

El OSI evalúa la relación entre el esfuerzo cortante vectorial promedio y su magnitud promedio. Valores altos indican cambios frecuentes en la dirección del flujo, creando un ambiente proaterogénico que favorece la disfunción endotelial, la inflamación local y el desarrollo de aterosclerosis.

Este análisis es particularmente relevante en pacientes post-TAVI para evaluar las alteraciones hemodinámicas inducidas por el implante valvular y su potencial impacto en la progresión de la enfermedad cardiovascular.

In [ ]:
importlib.reload(osi)

# Calculate OSI for all patients
all_osi_values = {}

for patient_id in tqdm(PATIENT_IDS, desc="Calculating OSI"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached OSI values
    cached_osi = load.load_metric_cache(patient_id, "osi_values", CACHE_DIR)

    if cached_osi is not None:
        all_osi_values[patient_id] = cached_osi
        continue

    # Calculate OSI if not cached
    patient_timesteps = list(all_patient_data[patient_id]["timesteps"].keys())
    patient_timestep_data = all_patient_data[patient_id]["timesteps"]
    patient_biomodel = all_patient_data[patient_id]["biomodel"]

    osi_values = osi.calculate_osi(
        patient_timesteps, patient_timestep_data, patient_biomodel, mu
    )

    # Store in analysis dictionary
    all_osi_values[patient_id] = osi_values

    # Cache the computed metric
    load.save_metric_cache(patient_id, "osi_values", osi_values, CACHE_DIR)

In [ ]:
# Create visualization
fig, axes = plt.subplots(len(PATIENT_IDS), 2, figsize=(12, 4 * len(PATIENT_IDS)))
if len(PATIENT_IDS) == 1:
    axes = axes.reshape(1, -1)

fig.suptitle("Multi-Patient OSI Analysis", fontsize=16, fontweight="bold")

for idx, patient_id in enumerate(PATIENT_IDS):
    if patient_id in all_osi_values:
        osi_values = all_osi_values[patient_id]

        # Histogram of OSI values
        axes[idx, 0].hist(
            osi_values,
            bins=50,
            color=patient_colors[patient_id],
            alpha=0.7,
            edgecolor="black",
        )
        axes[idx, 0].set_title(f"{patient_id} - OSI Distribution")
        axes[idx, 0].set_xlabel("OSI")
        axes[idx, 0].set_ylabel("Frequency")
        axes[idx, 0].grid(True, alpha=0.3)

        # Box plot of OSI values
        axes[idx, 1].boxplot(
            osi_values,
            patch_artist=True,
            boxprops={"facecolor": patient_colors[patient_id], "alpha": 0.7},
        )
        axes[idx, 1].set_title(f"{patient_id} - OSI Statistics")
        axes[idx, 1].set_ylabel("OSI")
        axes[idx, 1].grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0, 1, 0.98])  # Leave 4% space at top for suptitle
plt.show()

# Calculate and print statistics
# patient_stats, cross_patient_stats = osi.calculate_osi_statistics(
#     PATIENT_IDS, all_osi_values
# )
# osi.print_osi_summary(patient_stats, cross_patient_stats)

## ⚡ **Pérdida de energía viscosa**

La pérdida de energía viscosa (*Viscous Energy Loss*, VEL) cuantifica la disipación de energía mecánica del flujo sanguíneo debido a las fuerzas viscosas. Este parámetro es fundamental para evaluar la eficiencia hemodinámica y detectar anomalías en el flujo post-TAVI.

**Definición matemática:**

Para un fluido newtoniano incompresable, la función de disipación viscosa se define mediante el tensor de deformación:

$$
\Phi = \mu \sum_{i,j} \left( \frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \right)^2 = 2\mu \sum_{i,j} S_{ij}^2
$$

**Donde:**
- $\Phi$ = función de disipación viscosa [W/m³]
- $\mu$ = viscosidad dinámica de la sangre (≈ 0.004 Pa·s)
- $u_i$ = componentes de velocidad [m/s]
- $x_j$ = coordenadas espaciales [m]
- $S_{ij}$ = tensor de deformación simétrico [s⁻¹]

**Pérdida total de energía:**

La pérdida total se obtiene integrando la función de disipación sobre todo el volumen aórtico:

$$
E_{loss} = \int_V \Phi \, dV \quad [W]
$$

**Significado clínico:**

La VEL proporciona información sobre:
- **Eficiencia energética** del flujo sanguíneo
- **Presencia de turbulencias** locales
- **Impacto hemodinámico** del implante TAVI
- **Pérdidas de carga** en el sistema cardiovascular

Este análisis permite evaluar la calidad hemodinámica post-intervención y detectar regiones con disipación energética elevada que pueden comprometer la función cardiovascular.

In [ ]:
importlib.reload(ene)

# Parameters for viscous energy loss calculation
mu = 0.004  # Blood viscosity in Pa·s (typical value)

all_viscous_energy_loss = {}
all_total_dissipation = {}
all_avg_dissipation_rate = {}

# Process each patient
for patient_id in tqdm(PATIENT_IDS, desc="Processing patients"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached energy loss metrics
    cached_energy_loss = load.load_metric_cache(
        patient_id, "viscous_energy_loss", CACHE_DIR
    )
    cached_total_diss = load.load_metric_cache(
        patient_id, "total_dissipation", CACHE_DIR
    )
    cached_avg_diss = load.load_metric_cache(
        patient_id, "avg_dissipation_rate", CACHE_DIR
    )

    if (
        cached_energy_loss is not None
        and cached_total_diss is not None
        and cached_avg_diss is not None
    ):
        all_viscous_energy_loss[patient_id] = cached_energy_loss
        all_total_dissipation[patient_id] = cached_total_diss
        all_avg_dissipation_rate[patient_id] = cached_avg_diss
        continue

    # Calculate energy loss if not cached
    patient_data = all_patient_data[patient_id]
    patient_timesteps = list(patient_data["timesteps"].keys())
    patient_timestep_data = patient_data["timesteps"]
    patient_biomodel = patient_data["biomodel"]

    # Calculate energy loss for this patient
    energy_loss, total_diss, avg_diss = ene.calculate_patient_energy_loss(
        patient_timesteps, patient_timestep_data, patient_biomodel, mu
    )

    # Store in analysis dictionaries
    all_viscous_energy_loss[patient_id] = energy_loss
    all_total_dissipation[patient_id] = total_diss
    all_avg_dissipation_rate[patient_id] = avg_diss

    # Cache the computed metrics individually
    load.save_metric_cache(patient_id, "viscous_energy_loss", energy_loss, CACHE_DIR)
    load.save_metric_cache(patient_id, "total_dissipation", total_diss, CACHE_DIR)
    load.save_metric_cache(patient_id, "avg_dissipation_rate", avg_diss, CACHE_DIR)

In [ ]:
# Create main 2x2 subplot figure
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(
    "Multi-Patient Viscous Energy Loss Analysis", fontsize=16, fontweight="bold"
)

# Plot 1: Energy loss time series
ene.plot_energy_loss_time_series(
    axes[0, 0], PATIENT_IDS, all_viscous_energy_loss, patient_colors
)

# Plot 2: Average dissipation rate time series
ene.plot_dissipation_rate_time_series(
    axes[0, 1], PATIENT_IDS, all_avg_dissipation_rate, patient_colors
)

# Plot 3: Peak energy loss comparison
peak_energies = ene.calculate_peak_energies(PATIENT_IDS, all_viscous_energy_loss)
ene.plot_energy_comparison_bars(
    axes[1, 0],
    PATIENT_IDS,
    peak_energies,
    patient_colors,
    "Peak Energy Loss Comparison",
    "Peak Energy Loss (mW)",
)

# Plot 4: Mean energy loss comparison
mean_energies = ene.calculate_mean_energies(PATIENT_IDS, all_viscous_energy_loss)
ene.plot_energy_comparison_bars(
    axes[1, 1],
    PATIENT_IDS,
    mean_energies,
    patient_colors,
    "Mean Energy Loss Comparison",
    "Mean Energy Loss (mW)",
)

plt.tight_layout()
plt.show()

# Calculate comprehensive statistics
# patient_statistics = ene.calculate_energy_statistics(
#     PATIENT_IDS, all_viscous_energy_loss, all_flow_rates
# )

# # Print summary
# ene.print_energy_analysis_summary(
#     PATIENT_IDS, patient_statistics, all_viscous_energy_loss
# )

## 💨 **Gradientes de presión**

Los gradientes de presión representan la variación espacial de la presión sanguínea a lo largo de la geometría aórtica. Este parámetro hemodinámico fundamental permite evaluar las pérdidas de carga y detectar estenosis residuales o disfunciones valvulares post-TAVI.

**Definición matemática:**

El gradiente de presión se define como la derivada espacial del campo de presión:

$$
\nabla p = \frac{\partial p}{\partial x} \hat{i} + \frac{\partial p}{\partial y} \hat{j} + \frac{\partial p}{\partial z} \hat{k}
$$

**Donde:**
- $\nabla p$ = vector gradiente de presión [Pa/m]
- $p$ = campo de presión [Pa]
- $x, y, z$ = coordenadas espaciales [m]

**Métodos de cálculo:**

**1. Ecuación de Bernoulli simplificada:**

Para flujo estacionario e invíscido, la presión se relaciona con la velocidad mediante:

$$
p + \frac{1}{2}\rho v^2 = \text{constante}
$$

**2. Ecuación de Navier-Stokes completa:**

Para flujo viscoso e inestacionario, la ecuación de momentum incluye términos adicionales:

$$
\rho \left( \frac{\partial \vec{v}}{\partial t} + \vec{v} \cdot \nabla \vec{v} \right) = -\nabla p + \mu \nabla^2 \vec{v}
$$

**Parámetros físicos:**
- $\rho$ = densidad de la sangre (≈ 1050 kg/m³)
- $\mu$ = viscosidad dinámica de la sangre (≈ 0.004 Pa·s)
- $\vec{v}$ = vector velocidad del flujo [m/s]

**Significado clínico:**

- **Gradientes bajos (< 20 mmHg)**: Función valvular normal, flujo sin obstrucciones
- **Gradientes moderados (20-40 mmHg)**: Estenosis leve a moderada
- **Gradientes altos (> 40 mmHg)**: Estenosis severa o disfunción valvular

**Metodología de análisis:**

1. **Reconstrucción del campo de presión**: Se calcula la presión en cada punto del dominio aórtico
2. **Cálculo de gradientes**: Se determinan las derivadas espaciales usando métodos de diferencias finitas
3. **Análisis temporal**: Se evalúa la evolución de los gradientes durante el ciclo cardíaco
4. **Comparación entre métodos**: Se contrastan los resultados de Bernoulli y Navier-Stokes

Este análisis permite identificar regiones con pérdidas de presión elevadas y evaluar la eficiencia hemodinámica del sistema cardiovascular post-intervención.

In [ ]:
importlib.reload(pre)

# Parameters for pressure gradient calculation
rho = 1050  # Blood density in kg/m³ (typical value)
mu = 0.004  # Blood viscosity in Pa·s (typical value)

all_pressure_gradients = {}
all_pressure_drops = {}

# Process each patient
for patient_id in tqdm(PATIENT_IDS, desc="Calculating pressure gradients"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached pressure gradient metrics
    cached_pressure_gradients = load.load_metric_cache(
        patient_id, "pressure_gradients", CACHE_DIR
    )
    cached_pressure_drops = load.load_metric_cache(
        patient_id, "pressure_drops", CACHE_DIR
    )

    if cached_pressure_gradients is not None and cached_pressure_drops is not None:
        all_pressure_gradients[patient_id] = cached_pressure_gradients
        all_pressure_drops[patient_id] = cached_pressure_drops
        continue

    # Calculate pressure gradients if not cached
    patient_data = all_patient_data[patient_id]
    patient_timesteps = patient_data["timesteps"]
    patient_biomodel = patient_data["biomodel"]

    # Calculate pressure gradients for this patient
    pressure_gradients, pressure_drops = pre.calculate_patient_pressure_gradients(
        patient_id, patient_timesteps, patient_biomodel, rho, mu
    )

    # Store in analysis dictionaries
    all_pressure_gradients[patient_id] = pressure_gradients
    all_pressure_drops[patient_id] = pressure_drops

    # Cache the computed metrics individually
    load.save_metric_cache(
        patient_id, "pressure_gradients", pressure_gradients, CACHE_DIR
    )
    load.save_metric_cache(patient_id, "pressure_drops", pressure_drops, CACHE_DIR)

In [ ]:
# Create main 2x2 subplot figure for pressure gradients
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Multi-Patient Pressure Gradient Analysis", fontsize=16, fontweight="bold")

# Plot 1: Maximum pressure gradient time series (Bernoulli)
pre.plot_pressure_gradient_time_series(
    axes[0, 0],
    PATIENT_IDS,
    all_pressure_drops,
    patient_colors,
    "Maximum Pressure Gradient (Bernoulli) by Patient",
    "Max Pressure Gradient (Pa/m)",
    "max_grad_bernoulli",
)

# Plot 2: Maximum pressure gradient time series (Navier-Stokes)
pre.plot_pressure_gradient_time_series(
    axes[0, 1],
    PATIENT_IDS,
    all_pressure_drops,
    patient_colors,
    "Maximum Pressure Gradient (Navier-Stokes) by Patient",
    "Max Pressure Gradient (Pa/m)",
    "max_grad_ns",
)

# Plot 3: Peak pressure gradient comparison (Bernoulli)
peak_gradients_bernoulli = pre.calculate_peak_pressure_gradients(
    PATIENT_IDS, all_pressure_drops, "max_grad_bernoulli"
)
pre.plot_pressure_gradient_comparison_bars(
    axes[1, 0],
    PATIENT_IDS,
    peak_gradients_bernoulli,
    patient_colors,
    "Peak Pressure Gradient Comparison (Bernoulli)",
    "Peak Pressure Gradient (Pa/m)",
)

# Plot 4: Peak pressure gradient comparison (Navier-Stokes)
peak_gradients_ns = pre.calculate_peak_pressure_gradients(
    PATIENT_IDS, all_pressure_drops, "max_grad_ns"
)
pre.plot_pressure_gradient_comparison_bars(
    axes[1, 1],
    PATIENT_IDS,
    peak_gradients_ns,
    patient_colors,
    "Peak Pressure Gradient Comparison (Navier-Stokes)",
    "Peak Pressure Gradient (Pa/m)",
)

plt.tight_layout()
plt.show()

## 🌪️ **Vorticidad**

La vorticidad ($\omega$) es una medida de la rotación local del fluido en cada punto del campo de velocidades. Este parámetro biomecánico es fundamental para identificar estructuras de flujo complejas como vórtices, remolinos y zonas de recirculación en el flujo sanguíneo aórtico.

**Definición matemática:**

La vorticidad se define como el rotacional (curl) del campo de velocidades:

$$
\vec{\omega} = \nabla \times \vec{v} = \begin{vmatrix}
\hat{i} & \hat{j} & \hat{k} \\
\frac{\partial}{\partial x} & \frac{\partial}{\partial y} & \frac{\partial}{\partial z} \\
v_x & v_y & v_z
\end{vmatrix}
$$

**Componentes de la vorticidad:**

$$
\omega_x = \frac{\partial v_z}{\partial y} - \frac{\partial v_y}{\partial z}
$$

$$
\omega_y = \frac{\partial v_x}{\partial z} - \frac{\partial v_z}{\partial x}
$$

$$
\omega_z = \frac{\partial v_y}{\partial x} - \frac{\partial v_x}{\partial y}
$$

**Magnitud de la vorticidad:**

$$
|\vec{\omega}| = \sqrt{\omega_x^2 + \omega_y^2 + \omega_z^2}
$$

**Donde:**
- $\vec{\omega}$ = vector vorticidad [s⁻¹]
- $\vec{v}$ = vector velocidad [m/s]
- $v_x, v_y, v_z$ = componentes de velocidad [m/s]

**Significado físico:**

La vorticidad representa la tendencia de las partículas de fluido a rotar alrededor de un eje instantáneo. En el contexto del flujo aórtico post-TAVI:

- **Vorticidad baja (< 50 s⁻¹)**: Flujo laminar ordenado, condiciones hemodinámicas óptimas
- **Vorticidad moderada (50-200 s⁻¹)**: Transición hacia flujo turbulento, estructuras de vórtice
- **Vorticidad alta (> 200 s⁻¹)**: Flujo altamente turbulento, posible disfunción valvular

**Interpretación clínica:**

La presencia de alta vorticidad en regiones específicas puede indicar:
- **Estenosis residual** o mal funcionamiento de la válvula TAVI
- **Formación de vórtices** que pueden favorecer la hemólisis
- **Turbulencia excesiva** asociada con pérdidas energéticas
- **Zonas de estasis** propensas a la formación de trombos

Este análisis permite evaluar la calidad hemodinámica post-TAVI y detectar patrones de flujo anómalos que requieren seguimiento clínico.

In [ ]:
importlib.reload(vor)

# Calculate multi-patient vorticity
all_vorticity_data = {}
all_vorticity_magnitude = {}
all_max_vorticity = {}
all_avg_vorticity = {}

for patient_id in tqdm(PATIENT_IDS, desc="Processing patients"):
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        continue

    # Try to load cached vorticity metrics
    cached_vort_data = load.load_metric_cache(patient_id, "vorticity_data", CACHE_DIR)
    cached_vort_mag = load.load_metric_cache(
        patient_id, "vorticity_magnitude", CACHE_DIR
    )
    cached_max_vort = load.load_metric_cache(patient_id, "max_vorticity", CACHE_DIR)
    cached_avg_vort = load.load_metric_cache(patient_id, "avg_vorticity", CACHE_DIR)

    if (
        cached_vort_data is not None
        and cached_vort_mag is not None
        and cached_max_vort is not None
        and cached_avg_vort is not None
    ):
        all_vorticity_data[patient_id] = cached_vort_data
        all_vorticity_magnitude[patient_id] = cached_vort_mag
        all_max_vorticity[patient_id] = cached_max_vort
        all_avg_vorticity[patient_id] = cached_avg_vort
        continue

    # Calculate vorticity for single patient
    vort_data, vort_mag, max_vort, avg_vort = vor.calculate_patient_vorticity(
        patient_id, all_patient_data[patient_id]
    )

    # Store results
    all_vorticity_data[patient_id] = vort_data
    all_vorticity_magnitude[patient_id] = vort_mag
    all_max_vorticity[patient_id] = max_vort
    all_avg_vorticity[patient_id] = avg_vort

    # Cache the computed metrics individually
    load.save_metric_cache(patient_id, "vorticity_data", vort_data, CACHE_DIR)
    load.save_metric_cache(patient_id, "vorticity_magnitude", vort_mag, CACHE_DIR)
    load.save_metric_cache(patient_id, "max_vorticity", max_vort, CACHE_DIR)
    load.save_metric_cache(patient_id, "avg_vorticity", avg_vort, CACHE_DIR)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Multi-Patient Vorticity Analysis", fontsize=16, fontweight="bold")

# Plot 1: Maximum vorticity time series
vor.plot_vorticity_time_series(
    axes[0, 0],
    PATIENT_IDS,
    all_max_vorticity,
    patient_colors,
    "Maximum Vorticity by Timestep",
    "Max Vorticity (s⁻¹)",
)

# Plot 2: Average vorticity time series
vor.plot_vorticity_time_series(
    axes[0, 1],
    PATIENT_IDS,
    all_avg_vorticity,
    patient_colors,
    "Average Vorticity by Timestep",
    "Avg Vorticity (s⁻¹)",
)

# Plot 3: Peak maximum vorticity comparison
peak_values = vor.calculate_peak_vorticity_by_patient(PATIENT_IDS, all_max_vorticity)
vor.plot_vorticity_comparison_bars(
    axes[1, 0],
    peak_values,
    patient_colors,
    "Peak Maximum Vorticity Comparison",
    "Peak Max Vorticity (s⁻¹)",
)

# Plot 4: Average maximum vorticity comparison
avg_values = vor.calculate_average_vorticity_by_patient(PATIENT_IDS, all_max_vorticity)
vor.plot_vorticity_comparison_bars(
    axes[1, 1],
    avg_values,
    patient_colors,
    "Average Maximum Vorticity Comparison",
    "Avg Max Vorticity (s⁻¹)",
)

plt.tight_layout()
plt.show()

# Calculate and print statistics
# patient_stats, cross_patient_stats = vor.calculate_vorticity_statistics(
#     PATIENT_IDS, all_max_vorticity, all_avg_vorticity, all_flow_rates
# )

# vor.print_vorticity_summary(patient_stats, cross_patient_stats)

## 🌀 **Helicidad**

La helicidad (*H*) es un invariante topológico que cuantifica la correlación entre el campo de velocidades y su vorticidad asociada. Este parámetro biomecánico avanzado caracteriza la naturaleza helicoidal del flujo sanguíneo y es fundamental para identificar estructuras de flujo tridimensionales complejas en el sistema cardiovascular post-TAVI.

**Definición matemática:**

La helicidad se define como el producto escalar entre el vector velocidad y el vector vorticidad:

$$
H = \vec{v} \cdot \vec{\omega} = \vec{v} \cdot (\nabla \times \vec{v})
$$

**Donde:**
- *H* = helicidad local [m²/s²]
- $\vec{v}$ = vector velocidad del flujo [m/s]
- $\vec{\omega}$ = vector vorticidad [s⁻¹]

**Helicidad volumétrica integrada:**

Para caracterizar el comportamiento global del flujo aórtico, se calcula la helicidad total:

$$
H_{total} = \int_V \vec{v} \cdot \vec{\omega} \, dV
$$

**Interpretación física:**

La helicidad proporciona información sobre la topología tridimensional del flujo:

- **H > 0**: Flujo helicoidal dextrógiro (rotación en sentido horario vista desde la dirección del flujo)
- **H < 0**: Flujo helicoidal levógiro (rotación en sentido antihorario)
- **H ≈ 0**: Flujo bidimensional o ausencia de correlación velocidad-vorticidad

**Significado biomecánico:**

En el contexto del flujo aórtico, la helicidad revela:

- **Estructuras helicoidales**: Vórtices tubulares y flujos en espiral característicos del flujo fisiológico
- **Complejidad topológica**: Grado de tridimensionalidad y organización espacial del flujo
- **Eficiencia energética**: Los flujos helicoidales pueden ser más eficientes energéticamente que los turbulentos
- **Estabilidad del flujo**: Las estructuras helicoidales tienden a ser más estables que los vórtices planares

In [ ]:
importlib.reload(hel)

# Calculate helicity for all patients
all_helicity_data = {}
all_helicity_magnitude = {}
all_max_helicity = {}
all_avg_helicity = {}
all_abs_avg_helicity = {}

for patient_id in PATIENT_IDS:
    if patient_id not in all_patient_data:
        print(f"Warning: No data found for patient {patient_id}")
        # Initialize empty dictionaries for missing patients
        all_helicity_data[patient_id] = {}
        all_helicity_magnitude[patient_id] = {}
        all_max_helicity[patient_id] = {}
        all_avg_helicity[patient_id] = {}
        all_abs_avg_helicity[patient_id] = {}
        continue

    # Try to load cached helicity metrics
    cached_hel_data = load.load_metric_cache(patient_id, "helicity_data", CACHE_DIR)
    cached_hel_mag = load.load_metric_cache(patient_id, "helicity_magnitude", CACHE_DIR)
    cached_max_hel = load.load_metric_cache(patient_id, "max_helicity", CACHE_DIR)
    cached_avg_hel = load.load_metric_cache(patient_id, "avg_helicity", CACHE_DIR)
    cached_abs_avg_hel = load.load_metric_cache(
        patient_id, "abs_avg_helicity", CACHE_DIR
    )

    if (
        cached_hel_data is not None
        and cached_hel_mag is not None
        and cached_max_hel is not None
        and cached_avg_hel is not None
        and cached_abs_avg_hel is not None
    ):
        all_helicity_data[patient_id] = cached_hel_data
        all_helicity_magnitude[patient_id] = cached_hel_mag
        all_max_helicity[patient_id] = cached_max_hel
        all_avg_helicity[patient_id] = cached_avg_hel
        all_abs_avg_helicity[patient_id] = cached_abs_avg_hel
        continue

    # Calculate helicity if not cached
    patient_data = all_patient_data[patient_id]

    # Check if vorticity data is available for this patient
    if patient_id not in all_vorticity_data:
        print(
            f"Warning: Vorticity data not available for {patient_id}, skipping helicity calculation"
        )
        # Initialize empty dictionaries for this patient
        all_helicity_data[patient_id] = {}
        all_helicity_magnitude[patient_id] = {}
        all_max_helicity[patient_id] = {}
        all_avg_helicity[patient_id] = {}
        all_abs_avg_helicity[patient_id] = {}
        continue

    patient_vorticity_data = all_vorticity_data[patient_id]

    hel_data, hel_mag, max_hel, avg_hel, abs_avg_hel = hel.calculate_patient_helicity(
        patient_id, patient_data, patient_vorticity_data
    )

    # Store results
    all_helicity_data[patient_id] = hel_data
    all_helicity_magnitude[patient_id] = hel_mag
    all_max_helicity[patient_id] = max_hel
    all_avg_helicity[patient_id] = avg_hel
    all_abs_avg_helicity[patient_id] = abs_avg_hel

    # Cache the computed metrics individually
    load.save_metric_cache(patient_id, "helicity_data", hel_data, CACHE_DIR)
    load.save_metric_cache(patient_id, "helicity_magnitude", hel_mag, CACHE_DIR)
    load.save_metric_cache(patient_id, "max_helicity", max_hel, CACHE_DIR)
    load.save_metric_cache(patient_id, "avg_helicity", avg_hel, CACHE_DIR)
    load.save_metric_cache(patient_id, "abs_avg_helicity", abs_avg_hel, CACHE_DIR)

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Multi-Patient Helicity Analysis", fontsize=16, fontweight="bold")

# Plot 1: Maximum absolute helicity time series
hel.plot_helicity_time_series(
    axes[0, 0],
    PATIENT_IDS,
    all_max_helicity,
    patient_colors,
    "Maximum Absolute Helicity by Patient",
    "Max |Helicity| (mm²/s²)",
)

# Plot 2: Average helicity time series
hel.plot_helicity_time_series(
    axes[0, 1],
    PATIENT_IDS,
    all_avg_helicity,
    patient_colors,
    "Average Helicity by Patient",
    "Avg Helicity (mm²/s²)",
    add_zero_line=True,
)

# Plot 3: Peak maximum helicity comparison
hel.plot_helicity_comparison_bars(
    axes[1, 0],
    PATIENT_IDS,
    all_max_helicity,
    patient_colors,
    "Peak Maximum Helicity Comparison",
    "Peak Max Helicity (mm²/s²)",
)

# Plot 4: Average absolute helicity time series
hel.plot_helicity_time_series(
    axes[1, 1],
    PATIENT_IDS,
    all_abs_avg_helicity,
    patient_colors,
    "Average Absolute Helicity by Patient",
    "Avg |Helicity| (mm²/s²)",
)

plt.tight_layout()
plt.show()

# Calculate and print statistics
# patient_stats = hel.calculate_helicity_statistics(
#     PATIENT_IDS, all_max_helicity, all_avg_helicity, all_abs_avg_helicity
# )

# hel.print_helicity_summary(
#     PATIENT_IDS, patient_stats, all_max_helicity, all_flow_rates, all_max_vorticity
# )